In [ ]:
from azureml.core import Workspace, Experiment

"""
NOTE: CHANGE THIS EACH TIME
Workspace name: quick-starts-ws-123409
Azure region: southcentralus
Subscription id: 572f8abf-a1a0-4b78-8c6d-3630739c72b5
Resource group: aml-quickstarts-123409                  
"""

ws = Workspace.get(name="quick-starts-ws-123409")
exp1 = Experiment(workspace=ws, name="nuria-hyper")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp1.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_name = "nuria-p1"
vm_size = "Standard_D2_V2"
cluster_type = 0
try:
    aml_compute = ComputeTarget(workspace=ws, name=compute_name)
    print('Using existing compute target.')
except ComputeTargetException:
    print('Creating compute target.')
    cluster_type = 1
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    compute_target.wait_for_completion(show_output=True)


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

In [ ]:
# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
### C and max_iter according to train.py given
ps = RandomParameterSampling({
        "C": uniform(0.01, 2.0),
        "max_iter": choice(50,75,100,150,200)})

In [ ]:
# Specify a Policy
# policy = ### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.2, evaluation_interval = 1, delay_evaluation = 5)

In [ ]:
if "training" not in os.listdir():
    os.mkdir("./training")

In [ ]:
# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###
est = SKLearn(source_directory='.', 
              compute_target=compute_name, 
              entry_script='train.py')

In [ ]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(estimator=est,  
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,        
                             max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
hyper_run = exp1.submit(hyperdrive_config)
RunDetails(hyper_run).show()
hyper_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.
### YOUR CODE HERE ###
best_run = hyper_run.get_best_run_by_primary_metric()   
parameters = best_run.get_details()['runDefinition']['arguments']
joblib.dump(parameters, filename='outputs/best-hyperdrive.joblib')

In [ ]:
model = best_run.register_model(model_name='best-hyperdrive', model_path='.')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
# x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(ds) 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
train_data = x_train.join(y_train)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
exp2 = Experiment(ws, 'nuria-automl')  
auto_run = exp2.submit(automl_config, show_output = True)

In [ ]:
# Retrieve and save your best automl model.
### YOUR CODE HERE ###
best_run, fitted_model = auto_run.get_output()
joblib.dump(fitted_model, filename='outputs/best-automl.joblib')


In [ ]:
# register the best model
model = auto_run.register_model(model_name='best-automl')

In [ ]:
# RUN THIS AT THE END, AFTER CHECKING REGISTERED MODELS AND EVERYTHING ELSE

# Delete cluster
if(cluster_type == 0):
    aml_compute.delete()
else:
    compute_target.delete()